<a href="https://colab.research.google.com/github/nrubian/azure4everyone-samples/blob/master/Colaboratory_challengge_n5_now.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Configuración ambiente Spark

In [1]:
!apt-get install openjdk-8-jdk-headless --quiet > /dev/null
!wget --quiet !wget https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
!tar xf spark-3.3.2-bin-hadoop3.tgz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"
!pip install --quiet findspark
import findspark
findspark.init()
from pyspark.sql import*
from pyspark import SparkContext, SparkConf

In [2]:
sc = SparkContext("local", "Challenge_n5_now")
sc

<SparkContext master=local appName=Challenge_n5_now>

## Librerias

In [7]:
import pandas as pd


## Ruta raiz archivos
Para modificar el directorio en el cual esten los datasets

In [5]:
raiz= '/content/'

## Lectura de datasets con Pandas
Lectura con pandas y mostrar la ultima fila para validar preliminarmente el importe correcto

In [32]:
df_country_latest=pd.read_csv(raiz+"country_wise_latest.csv",sep=",")
df_country_latest.tail(1)

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
186,Zimbabwe,2704,36,542,2126,192,2,24,1.33,20.04,6.64,1713,991,57.85,Africa


In [27]:
df_covid_clean=pd.read_csv(raiz+"covid_19_clean_complete.csv",sep=",")
df_covid_clean.tail(1)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
49067,NaN,Lesotho,-29.61,28.2336,2020-07-27,505,12,128,365,Africa


In [28]:
df_day_wise=pd.read_csv(raiz+"day_wise.csv",sep=",")
df_day_wise.tail(1)

,Date,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,No. of countries
187,2020-07-27,16480485,654036,9468087,6358362,228693,5415,174623,3.97,57.45,6.91,187


In [29]:
df_full_grouped=pd.read_csv(raiz+"full_grouped.csv",sep=",")
df_full_grouped.tail(1)

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,WHO Region
35155,2020-07-27,Zimbabwe,2704,36,542,2126,192,2,24,Africa


In [30]:
df_usa_county_wise=pd.read_csv(raiz+"usa_county_wise.csv",sep=",")
df_usa_county_wise.tail(1)


,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed,Deaths
627919,84070020,US,USA,840,NaN,Weber-Morgan,Utah,US,41.27116,-111.914512,"Weber-Morgan, Utah, US",7/27/20,2375,24


In [31]:
df_worldometer_data=pd.read_csv(raiz+"worldometer_data.csv",sep=",")
df_worldometer_data.tail(1)

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
208,Western Sahara,Africa,598682.0,10,NaN,1.0,NaN,8.0,NaN,1.0,NaN,17.0,2.0,NaN,NaN,Africa


## Lectura de datasets con Spark
Leer RDD de cada dataset, y mostrar 1 registro para validar data

In [33]:
rdd_country_latest=sc.textFile(raiz+"country_wise_latest.csv")
rdd_country_latest.take(1)

['Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region']

In [34]:
rdd_covid_cleant=sc.textFile(raiz+"covid_19_clean_complete.csv")
rdd_covid_cleant.take(1)

['Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region']

In [35]:
rdd_day_wise=sc.textFile(raiz+"day_wise.csv")
rdd_day_wise.take(1)

['Date,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,No. of countries']

In [36]:
rdd_full_grouped=sc.textFile(raiz+"full_grouped.csv")
rdd_full_grouped.take(1)

['Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,WHO Region']

In [37]:
rdd_usa_county_wise=sc.textFile(raiz+"usa_county_wise.csv")
rdd_usa_county_wise.take(1)

['UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed,Deaths']

In [38]:
rdd_worldometer_data=sc.textFile(raiz+"worldometer_data.csv")
rdd_worldometer_data.take(1)

['Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region']

## Información importante de los datos
**Información general**
- Información referente a las cifras del covid como, muertes, recuperados, activos, confirmados, nuevos casos etc.

**country_wise_lastest:**
- Este Dataset trae datos del último día para este caso Julio 27 de 2020

**Covid Clean:**
- Datos  por estado , tener en cuenta que hay gran cantidad de vacios en el estado, Datos día a día 

**day wise:** 
- Datos agrupados por fecha sin detalle por país

**Full Group:** 
- Son los datos de covid Clean, sin estado, agrupados fecha y país

**usa_county_wise:** 
- Datos de EU, que contiene datos con nivel de detalle de admin2 presuntamente neighborhood

**worldometer_data:** 
- No se logra identificar a que hace referencia este dataset ya que no coinciden las cifras con los demas, tiene nivel de detalle por país, adicional se evidencia que trae mas paises que los otros datasets y los nombres de algunos estan diferentes

## Implementación del Diagrama Entidad Relación propuesto

In [40]:
#Función que separa por "," cada campo y toma los campos Admin2,Country_Region necesarios para crear esta tabla
def select_columns(linea):
    UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,otros=linea.split(",",8)
    return (Admin2,Country_Region)

d_admin2=rdd_usa_county_wise.map(select_columns).distinct()

In [44]:
d_admin2= d_admin2.map(lambda row: ','.join([str(elem) for elem in row]))
d_admin2.saveAsPickleFile(raiz+"d_admin2")